# 清洗104人力銀行職缺的"職缺內容"資料，使其適合模型訓練使用

包含下列步驟 
> 字串清洗

> 資串整合

> Jieba結巴斷詞

#### 效果展示

清洗前: 

    1.負責介紹及銷售商品。
    2.提供顧客之接待與需求服務（如：電話諮詢、調貨、修改、包裝及退換貨處理）。
    3.負責商品進貨入庫、銷售管理及庫存管理。
    4.負責商品包裝、陳列及促銷品換檔工作。
    5.維持店櫃週遭整潔。
    6.提升業績。
    需符合以下條件，無經驗勿投履歷~
    1.具備少淑女服飾一年以上銷售經驗
    2.充滿活力並有服務熱忱

清洗後: 

    介紹/銷售/商品/顧客/接待/服務/電話/諮詢/調貨/包裝/退換貨/商品/進貨/入庫/銷售/管理/庫存/管理/商品/包裝/陳列/促銷品/換檔/維持/店櫃/週遭/整潔/提升/業績/淑女/服飾/銷售/服務/門市/店員/專櫃

## 1. read data from MongoDB

In [1]:
import pymongo
from secrets import HOST

In [2]:
def mongo_connect_build(db_name, col_name):
    
    global mycol
    client = pymongo.MongoClient(HOST) # 跟mongodb建立連

    db = client[db_name] 
    mycol = db[col_name] 
    
def data_find():
    # 尋找多筆資料，排除 id
    return mycol.find({}, {"_id":0})

In [3]:
# connect to mongoDB to get data
mongo_connect_build('Topic_104', 'job_clean_test') 
lst_jobdata = [i for i in data_find()] # store the data to list

In [4]:
print('所有資料數量: ', len(lst_jobdata))

所有資料數量:  135384


顯示兩筆清洗前資料

In [5]:
# show job description data before cleaning
for i in lst_jobdata[0:2]:
    print(i['jobDescription'])
    print(i['other'])
    print("-"*20)

1.負責介紹及銷售商品。
2.提供顧客之接待與需求服務（如：電話諮詢、調貨、修改、包裝及退換貨處理）。
3.負責商品進貨入庫、銷售管理及庫存管理。
4.負責商品包裝、陳列及促銷品換檔工作。
5.維持店櫃週遭整潔。
6.提升業績。
需符合以下條件，無經驗勿投履歷~
1.具備少淑女服飾一年以上銷售經驗
2.充滿活力並有服務熱忱
--------------------
歐德傢俱連鎖事業鼓勵對系統傢俱設計有濃厚的有志之士加入，透過完整的晉升制度及各項多元職系發展，讓同仁在歐德集團開創自己的事業。
本職務有業務經驗尤佳，以設計、銷售系統傢俱為主，依業主需求從系統介紹、工地丈量、繪圖、監工等..為業主服務，讓您走向專業發展的設計領域，歡迎您加入歐德集團。
自備交通工具
--------------------


## 2. Clean job Description data

In [6]:
import re

### a. 定義資料清洗 & 欄位整合 function 

In [7]:
def string_clean(jobDescription): 
    '''
    input: string (original job description)
    output: string (clean job description)
    '''
    job_desc = jobDescription.split('\n') # 根據換行符號轉乘 List格式
    job_words = ''
    
    for words in job_desc:
        words = words.replace('\t', ' ').replace('\r', ' ')
        words = re.sub(r'[^\w\s]',' ', words)  # remove all punctuations
        words = re.sub(r'\d+', ' ', words)  # remove all numbers
        words = words.strip() # remove white space
        words += ' '
        job_words += words
        
    return (job_words)

#### 資料清洗整合

In [8]:
# 將不同欄位的資料清洗並整合

def concate_clean_data(data):
    '''
    input: 
        data: a dictionary of a job
    output
        jobDescription_clean: a string of a job description after cleaning
    '''
    
    jobDescription = data['jobDescription']
    jobOther = data['other']
    
    job_words = string_clean(jobDescription) # clean string
    job_words_other = string_clean(jobOther) # clean string

    jobDescription_clean = job_words + job_words_other # add two string
    
    # add jobCategory
    jobDescription_clean = jobDescription_clean + " " + data['jobCategory'].replace('／', ' ')
    
    def add_column_data(colname, jobDescription_clean):
        if len(data[colname]) > 0:
            for j in data[colname]:
                jobDescription_clean = jobDescription_clean + " " + j
    
    add_column_data('major_clean', jobDescription_clean)
    add_column_data('language_clean', jobDescription_clean)
    add_column_data('skill_clean', jobDescription_clean)
    add_column_data('certificate_clean', jobDescription_clean)
    add_column_data('specialty', jobDescription_clean)
            
    return jobDescription_clean
    

顯示三兩筆清洗整合後資料

In [9]:
for i in lst_jobdata[0:3]:
    data = concate_clean_data(i)
    if data != None:
        print(data)
        print("-"*20)

負責介紹及銷售商品 提供顧客之接待與需求服務 如 電話諮詢 調貨 修改 包裝及退換貨處理 負責商品進貨入庫 銷售管理及庫存管理 負責商品包裝 陳列及促銷品換檔工作 維持店櫃週遭整潔 提升業績 需符合以下條件 無經驗勿投履歷 具備少淑女服飾一年以上銷售經驗 充滿活力並有服務熱忱  門市 店員 專櫃人員
--------------------
歐德傢俱連鎖事業鼓勵對系統傢俱設計有濃厚的有志之士加入 透過完整的晉升制度及各項多元職系發展 讓同仁在歐德集團開創自己的事業 本職務有業務經驗尤佳 以設計 銷售系統傢俱為主 依業主需求從系統介紹 工地丈量 繪圖 監工等  為業主服務 讓您走向專業發展的設計領域 歡迎您加入歐德集團 自備交通工具  國內業務人員
--------------------
熟悉成本會計 熟悉三角貿易會計 進貨 銷貨 費用單據審核及傳票製作 零用金異動及撥補 應收 應付帳務處理 會計報表編製 有操作系統經驗者佳 採購與庫存管理 進銷存管理 與廠商接洽訂單事宜 須熟悉正航會計軟體 能完成財務報表    公司福利 週休二日 特休 年假 依勞基法規定 須熟悉正航會計軟體  記帳 出納 一般會計
--------------------


### b. 使用 Jieba 分詞

In [10]:
import jieba

In [11]:
# 指定辭典檔
jieba.load_userdict('jieba_data/Jobcontent_dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.724 seconds.
Prefix dict has been built successfully.


In [12]:
# 使用結巴斷詞，產生 list of words

def isEnglish(s): # 檢查字元是否為英文
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True


def jieba_cut(data):  
    # 使用結巴斷詞
    seg_result = jieba.cut(data, cut_all=False)

    # 篩選斷詞，去掉單一中文字
    lst_seg = []
    
    for i in list(seg_result):
        i = i.strip()
        if len(i) < 1:  #排除空值
            continue
        elif i in stop_words: # 排除stopwords
            continue
        elif isEnglish(i) == False and len(i) == 1: #排除單一中文字
            continue
        elif i.isdigit() == True: #排除數字
            continue
        else:
            lst_seg.append(i)
   
    return lst_seg

## 3. 建立Job description 的 stopwords

In [13]:
# 載入創建好的stopword表單

with open(file='jieba_data/Jobcontent_stopwords.txt',mode='r', encoding="UTF-8") as file:
    stop_words = file.read().split('\n')
    stop_words = [i.strip() for i in stop_words]

In [14]:
# 建立stopwords用: 顯示字詞頻率，將不適合的字放入stopwords

dict_words_frequency = {}

for i in lst_jobdata[0:500]: # 取500筆展示效果
    data = concate_clean_data(i)
    if data != None:
        
        # 使用結巴斷詞
        lst_seg = jieba_cut(data)
        
        # 計算斷詞出現頻率
        for i in lst_seg:
            if i not in dict_words_frequency:
                dict_words_frequency[i] = 1
            else:
                dict_words_frequency[i] += 1

dict_words_frequency = {k: v for k, v in sorted(dict_words_frequency.items(), key=lambda item: item[1], reverse=True)}

print(dict_words_frequency) 

{'管理': 236, '服務': 218, '商品': 168, '設計': 150, '業務': 148, '開發': 146, '協助': 133, '系統': 130, '作業': 125, '產品': 124, '顧客': 122, '門市': 122, '客戶': 116, '工程': 103, '銷售': 102, '主管': 101, '維護': 99, '專案': 98, '規劃': 87, '設備': 85, '現場': 79, '溝通': 78, '清潔': 75, '證券': 72, '整理': 72, '證照': 70, '技術': 70, '收銀': 68, '專業': 67, '操作': 67, '分析': 64, '包裝': 63, '專櫃': 62, '店員': 61, '資訊': 61, '測試': 61, '陳列': 56, '軟體': 53, '保險': 53, '教育': 53, '行銷': 52, '辦理': 51, '諮詢': 50, '金融': 46, '網頁': 44, '餐飲': 44, '發展': 43, '資格': 43, '安全': 43, '協調': 42, '資料': 42, '出口': 40, '生產': 40, '企劃': 40, '市場': 39, '行業': 38, '語文': 38, '專員': 38, '行政': 38, '文件': 38, '核定': 37, '理財': 37, '飯店': 37, '介紹': 36, '興捷': 36, '便利商店': 36, '國小': 36, '旅遊': 36, '補貨': 36, 'experience': 36, '進貨': 35, '制度': 35, '機械': 35, '流程': 35, '盤點': 35, '網路': 35, 'management': 35, '品質': 34, '建立': 33, '業績': 32, '食材': 32, '程式': 32, '接待': 31, '保養': 31, '衛生': 31, '傢俱': 30, '事業': 30, '電腦': 30, '結帳': 30, '賣場': 30, 'C': 30, '作業員': 29, '整合': 29, '準備': 29, '細心': 29, '倉儲': 28, '機台':

## 4. 顯示使用stopwords之後的斷詞結果

顯示兩筆處理後效果

In [15]:
for i in lst_jobdata[0:2]:
    data = concate_clean_data(i)
    if data != None:
        
        # 使用結巴斷詞
        lst_seg = jieba_cut(data)
        
        # 顯示斷詞結果    
        str_result = '/'.join(lst_seg)
        print(str_result)
        print("="*30)

介紹/銷售/商品/顧客/接待/服務/電話/諮詢/調貨/包裝/退換貨/商品/進貨/入庫/銷售/管理/庫存/管理/商品/包裝/陳列/促銷品/換檔/維持/店櫃/週遭/整潔/提升/業績/淑女/服飾/銷售/服務/門市/店員/專櫃
傢俱/連鎖/事業/系統/傢俱/設計/制度/多元/發展/事業/業務/設計/銷售/系統/傢俱/業主/系統/介紹/工地丈量/繪圖/監工/業主/服務/走向/專業/發展/設計/交通工具/業務


## 5. 將結果寫入JSON檔

In [16]:
import json

In [17]:
def dump_json_file(query_dict, file_name):
    dumped_json_cache = json.dumps(query_dict)
    fw = open(file_name,"w")
    fw.write(dumped_json_cache)
    fw.close()
    print('successfully write down the file: ', file_name)

In [18]:
dump_json_file(lst_jobdata, 'json_data/JobDescription_Clean.json')

successfully write down the file:  json_data/JobDescription_Clean.json
